# Prompt engineering on worst econ questions: answer verification, temperature, red/blue, context

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from ollama_models import ollama_models
models = ollama_models()

In [3]:
models

['mistral-small3.2:24b-instruct-2506-q4_K_M']

In [4]:
model = models[0]

In [5]:
from load_saved_questions import load_saved_questions
from call_local_llm import call_local_llm

In [6]:
question = load_saved_questions([4830])[0]

In [7]:
    from community_forecast import community_forecast
    import pandas as pd
    from flatten_dict import flatten_dict
    from datetime import datetime
    from gather_research_and_set_prompt import gather_research_and_set_prompt
    from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
    from tqdm import tqdm
    import time

In [8]:
    print('START model', model, 'id', id)
    start_time = time.time()
    questions = [question]
    id = question.id_of_question
    id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}
    df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
    df['id_of_post'] = [q.id_of_post for q in questions]
    df['id_of_question'] = [q.id_of_question for q in questions]
    df['question_options'] = df['question_options'].apply(repr)
    df['today'] = datetime.now().strftime("%Y-%m-%d")
    df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)

START model mistral-small3.2:24b-instruct-2506-q4_K_M id <built-in function id>


In [9]:
    df1 = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title',
        'question_resolution_criteria', 'question_fine_print', 'question_type', 
         'question_description',
        'question_options', 'question_group_variable', 'question_question_weight',
        'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
        'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']].copy()

In [10]:
    df2, rag = gather_research_and_set_prompt(df1, True, model)

Loaded existing index from forecast_index.faiss
Index contains 5839 vectors at initialization


In [11]:
df3 = df2[['id_of_question', 'title',
        'question_resolution_criteria', 'question_fine_print', 'question_type', 
         'question_description',
        'question_options', 'question_group_variable', 'question_question_weight',
        'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
        'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd',
          'research', 'asknews', 'learning']].copy()

In [12]:
row = df3.iloc[0]

In [13]:
row

id_of_question                                                               4830
title                             What will the global fertility rate be in 2050?
question_resolution_criteria    The age-specific fertility rate is the number ...
question_fine_print                                                              
question_type                                                             numeric
question_description                                                             
question_options                                                             None
question_group_variable                                                          
question_question_weight                                                      1.0
question_unit                                                  children per woman
question_open_upper_bound                                                    True
question_open_lower_bound                                                   False
question_scaling

In [14]:
for item, value in row.items():
    print(item.upper())
    print()
    print(value)
    print('===============================')
    print()

ID_OF_QUESTION

4830

TITLE

What will the global fertility rate be in 2050?

QUESTION_RESOLUTION_CRITERIA

The age-specific fertility rate is the number of children born per woman per year for women at a specified age. Integrating over a lifetime gives a metric called the [total fertility rate](https://ourworldindata.org/fertility-rate#data-quality-definition), which is the average number of children that a woman would have if all the age-specific fertility rates stayed constant. 

Globally, the total fertility rate was [2.49 children per woman](https://ourworldindata.org/fertility-rate#50-years-ago-the-average-woman-had-five-children-since-then-the-number-has-halved) in 2015, down from 5.05 in 1950. This large decline is part of what is known as the [demographic transition](https://en.wikipedia.org/wiki/Demographic_transition) from high birth rates and high child mortality to low birth rates and low child mortality. The causes of this include more women in education and work, as well

In [15]:
row.question_resolution_criteria

"The age-specific fertility rate is the number of children born per woman per year for women at a specified age. Integrating over a lifetime gives a metric called the [total fertility rate](https://ourworldindata.org/fertility-rate#data-quality-definition), which is the average number of children that a woman would have if all the age-specific fertility rates stayed constant. \n\nGlobally, the total fertility rate was [2.49 children per woman](https://ourworldindata.org/fertility-rate#50-years-ago-the-average-woman-had-five-children-since-then-the-number-has-halved) in 2015, down from 5.05 in 1950. This large decline is part of what is known as the [demographic transition](https://en.wikipedia.org/wiki/Demographic_transition) from high birth rates and high child mortality to low birth rates and low child mortality. The causes of this include more women in education and work, as well as greater access to contraception. \n\nA [recent study](https://www.bbc.co.uk/news/health-53409521) by 

In [16]:
prompt = f"""Here is some information about a question:

TITLE
=====

{row.title}

RESOLUTION CRITERIA
===================

{row.question_resolution_criteria}

SCALE
=====

From {row.question_scaling_range_min} to {row.question_scaling_range_max}

Using the information, answer the following questions:

a.  What is the minimum value for the answer?
b.  What is the maximum value for the answer?
c.  Is there a formula for the answer given in the above text?
d.  If there is a formula, what needs to go into the formula and how do you estimate it?  Write a python function for the formula, commenting its inputs and outputs for clarity
"""

In [17]:
print(prompt)

Here is some information about a question:

TITLE
=====

What will the global fertility rate be in 2050?

RESOLUTION CRITERIA

The age-specific fertility rate is the number of children born per woman per year for women at a specified age. Integrating over a lifetime gives a metric called the [total fertility rate](https://ourworldindata.org/fertility-rate#data-quality-definition), which is the average number of children that a woman would have if all the age-specific fertility rates stayed constant. 

Globally, the total fertility rate was [2.49 children per woman](https://ourworldindata.org/fertility-rate#50-years-ago-the-average-woman-had-five-children-since-then-the-number-has-halved) in 2015, down from 5.05 in 1950. This large decline is part of what is known as the [demographic transition](https://en.wikipedia.org/wiki/Demographic_transition) from high birth rates and high child mortality to low birth rates and low child mortality. The causes of this include more women in educatio

In [18]:
terms = call_local_llm(prompt, model)

START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 2.6182870507240295


In [19]:
print(terms)

a. The minimum value for the answer is 1.33 (as predicted by the study for the year 2100, but no specific minimum is given for 2050).

b. The maximum value for the answer is 2.08 (as predicted by the study for the year 2100, but no specific maximum is given for 2050).

c. There is no explicit formula given in the text for the answer. The prediction is based on extrapolating trends in education and contraception access.

d. Since there is no explicit formula provided, I cannot write a Python function for the formula. However, if we were to model the decline in fertility rates based on the given data points (2.49 in 2015 and a range of 1.33-2.08 by 2100), we could create a simple linear interpolation function. Here's an example:

```python
def predict_fertility_rate(year):
    """
    Predicts the global total fertility rate for a given year based on linear interpolation between known data points.

    Inputs:
    year (int): The year for which the fertility rate is to be predicted.

   

In [20]:
prompt2 = f"""
You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
{row.title}
```

```news
{row.asknews}

```research
{row.research}
```

Using the title, news and research, make a forecast according to the formulas you derived in the previous prompt response.
If the Python function has multiple variables, assess each variable individually and then use the function to compute the measure.

The last thing you write is your final answer as this sequence of percentile levels in percent and values as floating point numbers without currency symbols, commas or spelled out numbers like "trillion", just the raw complete number:
"
Percentile 10: XX
Percentile 20: XX
Percentile 40: XX
Percentile 60: XX
Percentile 80: XX
Percentile 90: XX
"
Each line of the final answer MUST START with the word "Percentile".  For example if you have "10: 201" instead of "Percentile 10: 201", that is wrong.

PLEASE REMEMBER THAT THE ANSWER MUST BE IN THE TERMS REQUESTED BY THE PROBLEM.  
A QUICK CORRECTNESS CHECK IS THAT YOUR ANSWER MUST LIE BETWEEN THE MINIMUM AND MAXIMUM VALUES SPECIFIED.  
THE REQUESTED TERMS ARE AS FOLLOWS:

```terms
{terms}
```
"""

In [21]:
print(prompt2)


You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
What will the global fertility rate be in 2050?
```

```news
Here are the relevant news articles:

**10 Trends That Will Shape the Future of Food by 2050**
By 2050, the world's food system will be under pressure due to the need to reach net-zero emissions and a projected global population of over 10 billion people. To reduce food waste, companies will need to rethink the food chain. According to food futurists Morgaine Gaye and Joseph Poore, there will be 10 key trends in food production, consumption, and preparation. These include: 1) 'community meals' where food is shared and valued for its emotional and relational aspects; 2) the end of weekly grocery shopping, replaced by AI-powered meal planning and home delivery; 3) clothing that can grow food, such as shirts with built-in gardens; 4) urban agriculture, wit

In [22]:
answer_mistral = call_local_llm(prompt2, model)

answer_mistral2 = call_local_llm(prompt2, model)

answer_mistral3 = call_local_llm(prompt2, model)

START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 4.78366615374883
START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 4.60348748366038
START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 5.884805087248484


In [23]:
from extract_forecast import extract_percentile_numbers

In [24]:
forecasts = [extract_percentile_numbers(x) for x in [answer_mistral, answer_mistral2, answer_mistral3]]

In [25]:
forecasts

[{10: 1.33, 20: 1.43, 40: 1.6, 60: 1.8, 80: 1.95, 90: 2.08},
 {10: 1.76, 20: 1.8, 40: 1.88, 60: 1.96, 80: 2.0, 90: 2.08},
 {10: 1.94, 20: 2.0, 40: 2.09, 60: 2.15, 80: 2.24, 90: 2.3}]

In [26]:
from median_dictionaries import median_dictionaries

In [27]:
forecast = median_dictionaries(forecasts)

In [28]:
forecast

{10: 1.76, 20: 1.8, 40: 1.88, 60: 1.96, 80: 2.0, 90: 2.08}

In [29]:
prompt = f"""
Summarize the gist pf the rationale or thinking of the following answers from 3 different forecasters to a single problem. 

```forecast 1
{answer_mistral}
```

```forecast 2
{answer_mistral2}
```

```forecast 3
{answer_mistral3}
```

DO NOT REFER TO THE 3 FORECASTERS.  PRESENT THIS AS YOUR OWN THINKING, YOUR OWN RATIONALE.  Use as your final the median forecast which is

```median forecast
{forecast}
```
"""

In [30]:
rationale = call_local_llm(prompt, model)

START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 3.781203591823578


In [31]:
print(rationale)

To forecast the global total fertility rate (TFR) for the year 2050, I used a linear interpolation method based on known data points from 2015 and projected ranges for 2100. Here’s the rationale behind my approach:

### Step 1: Define the Known Data Points
- **2015**: TFR = 2.49
- **2100 (minimum)**: TFR = 1.33
- **2100 (maximum)**: TFR = 2.08

### Step 2: Calculate the Slopes and Intercepts for Linear Interpolation
For the minimum prediction:
- Slope (min) = (1.33 - 2.49) / (2100 - 2015) = -0.0156
- Intercept (min) = 2.49 - (-0.0156 * 2015) = 2.49 + 31.434 = 33.924

For the maximum prediction:
- Slope (max) = (2.08 - 2.49) / (2100 - 2015) = -0.0041
- Intercept (max) = 2.49 - (-0.0041 * 2015) = 2.49 + 8.2615 = 10.7515

### Step 3: Predict the TFR for 2050
Using the linear interpolation formulas:
- **Minimum TFR for 2050**: -0.0156 * 2050 + 33.924 = 1.76
- **Maximum TFR for 2050**: -0.0041 * 2050 + 10.7515 = 2.08

### Step 4: Generate Percentile Forecasts
Assuming a normal distribution 

In [32]:
row.forecast = rationale

In [33]:
row.prediction = forecast

In [34]:
row.crowd

{10: 1.2404005339653996,
 20: 1.471560916910481,
 40: 1.6992185491831817,
 60: 1.9151103347581657,
 80: 2.182339722200339,
 90: 2.3414068703959257}

In [ ]:
from error import error

error(row)